In [ ]:
from __future__ import print_function
%matplotlib nbagg

import datetime as dt
import os
import platform

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.dates import num2date, DateFormatter
from scipy.io import readsav
from sunpy.time import TimeRange
from sunpy.timeseries import TimeSeries
from sunpy.net import Fido, attrs

from getrstn import GetRSTN
from noaareport import NoaaReport

from utils import (
    onclick, posicao, calculo_de_indice, load_dados,
    calculo_da_media, CAMINHO_ABSOLUTO, ponto_mais_proximo,
    get_correct_goes_index
)

In [ ]:
dia = "18"
mes = "12"
ano = "2009"
df, time, diretorio = load_dados(dia, mes, ano)

In [ ]:
plt.rcParams['figure.figsize'] = 7,5
fig, axis = plt.subplots()

# Usa o index do dataframe para o eixo x.
axis.plot(df)
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
"""
 Ordem de seleção:
  Os dois últimos (últimos) selecionados são usados para o calculo das médias.
  Os dois penúltimos (primeiros) selecionados são usados para representar a média
  no grafico.
"""
dados_da_media = calculo_da_media(df)

indice_ig1 = dados_da_media[0]
indice_ig2 = dados_da_media[1]
medias = dados_da_media[2]
media_final_r = medias['R']
media_final_l = medias['L']
tempo1_flare = dados_da_media[3]
tempo2_flare = dados_da_media[4]

In [ ]:
fig_R, axis = plt.subplots()
# Usa o mesmo index do dataframe para o eixo x.
axis.plot(df["R"][tempo1_flare:tempo2_flare])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

fig_L, axis = plt.subplots()
# Usa o mesmo index do dataframe para o eixo x.
axis.plot(df["L"][tempo1_flare:tempo2_flare])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

In [ ]:
LM_normalizado = df["L_normalizado"][tempo1_flare:tempo2_flare]
RM_normalizado = df["R_normalizado"][tempo1_flare:tempo2_flare]

RL_V = df["R_normalizado"][tempo1_flare:tempo2_flare] - df["L_normalizado"][tempo1_flare:tempo2_flare]
RL_I = df["L_normalizado"][tempo1_flare:tempo2_flare] + df["R_normalizado"][tempo1_flare:tempo2_flare]

fig_IV, (axis, ax, ax2) = plt.subplots(3)

# Usa o mesmo index do dataframe para o eixo x.
axis.plot(LM_normalizado, label="L")
axis.plot(RM_normalizado, label="R")
axis.set_title("Analise 7 GHz")
axis.legend()
axis.set_ylabel("Flux (SFU)")
axis.yaxis.grid(color='gray')

ax.plot(RL_I)
ax.set_ylabel("I (SFU)")
ax.yaxis.grid(color='gray')

ax2.plot(RL_V)
ax2.set_ylabel("V (SFU)")
ax2.yaxis.grid(color='gray')

axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax2.xaxis.set_major_formatter(DateFormatter('%H:%M'))
pico_7ghz = max(RL_I)

# Salva o gráfico.
nome_do_arquivo = "I-V-Flux(SFU)"
if platform.system() == "Linux":
    fig_IV.savefig(CAMINHO_ABSOLUTO + diretorio + "/" + nome_do_arquivo)
else:
    fig_IV.savefig(CAMINHO_ABSOLUTO + diretorio + "\\" + nome_do_arquivo)

## Pico (só se tiver algum problema no gráfico)

In [ ]:
fig, axis = plt.subplots()

# Usa o index do dataframe para o eixo x.
axis.plot(df["R_normalizado"])
axis.plot(df["L_normalizado"])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
print(posicao[-1])
pico_7ghz = num2date(posicao[-1][0])

descarte, tempo_inicial = calculo_de_indice(RL_I, pico_7ghz)
pico_7ghz = RL_I[tempo_inicial]

## RSTN

In [ ]:
rstn = GetRSTN(dia, mes, ano, "dados_rstn")
rstn.download_file()
rstn.decompress_file()
rstn_data = rstn.create_dataframe()

In [ ]:
del rstn_data["f245"]
del rstn_data["f410"]
del rstn_data["f610"]
del rstn_data["f1415"]
# del rstn_data["f2695"]

In [ ]:
ax = rstn_data.plot()
fig = ax.get_figure()
fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
posicao_inicial_rstn = posicao[-2]
posicao_final_rstn = posicao[-1]

p1 = num2date(posicao_inicial_rstn[0])
p2 = num2date(posicao_final_rstn[0])

descarte, tempo_inicial = calculo_de_indice(rstn_data, p1)
descarte, tempo_final = calculo_de_indice(rstn_data, p2)

medias = calculo_da_media(rstn_data, True)

fig, axis = plt.subplots()

plt.title("RSTN Normalizado")
for column in rstn_data.columns:
    if len(column) > 11:
        axis.plot(rstn_data.index, rstn_data[column], label=column)
        

axis.xaxis.set_major_formatter(DateFormatter("%H:%M"))
plt.legend()

In [ ]:
posicao_inicial_rstn = posicao[-4]
posicao_final_rstn = posicao[-3]

p1 = num2date(posicao_inicial_rstn[0])
p2 = num2date(posicao_final_rstn[0])

descarte, tempo_inicial = calculo_de_indice(rstn_data, p1)
descarte, tempo_final = calculo_de_indice(rstn_data, p2)

In [ ]:
picos = {
    "2.695": 0,
    "4.995": 0,
    "7.0": pico_7ghz,
    "8.8": 0,
    "15.4": 0
}

fig, axis = plt.subplots()

plt.title("RSTN Normalizado")
for column in rstn_data.columns:
    if len(column) > 11:
        column_freq = column[1:5]
        if column_freq == "2695":
            picos["2.695"] = max(rstn_data[column][tempo_inicial:tempo_final])
        elif column_freq == "4995":
            picos["4.995"] = max(rstn_data[column][tempo_inicial:tempo_final])
        elif column_freq == "8800":
            picos["8.8"] = max(rstn_data[column][tempo_inicial:tempo_final])
        elif column_freq == "1540":
            picos["15.4"] = max(rstn_data[column][tempo_inicial:tempo_final])
        axis.plot(rstn_data.index[tempo_inicial:tempo_final], rstn_data[column][tempo_inicial:tempo_final], label=column)

print(picos)
axis.xaxis.set_major_formatter(DateFormatter("%H:%M"))
plt.legend()

In [ ]:
# Pega o indice do tempo do pico no 7ghz para pegar o
# indice no rstn, comparando os horários.

for index, item in enumerate(RL_I):
    if item == pico_7ghz:
        indice_7ghz = index

RL_I.index[indice_7ghz]

tempo_proximo_rstn = ponto_mais_proximo(RL_I.index, RL_I.index[indice_7ghz])
tempo_proximo_rstn = str(tempo_proximo_rstn)[:19]

for index, item in enumerate(rstn_data["f8800_normalizado"]):
    tempo_rstn = str(rstn_data.index[index])[:19]
    if tempo_proximo_rstn == tempo_rstn:
        print(RL_I.index[indice_7ghz])
        print(rstn_data.index[index])
        horario_pico = rstn_data.index[index]
        indice_rstn = index


## Espectro

In [ ]:
# Colocar as frequencias em ordem.
picos_ordenados = [float(x) for x in picos.keys()]

fig_espectro, axis = plt.subplots()

for column in picos_ordenados:
    axis.plot(column, picos[str(column)], '-o', label=column)

axis.set_xlabel("Frequência")
axis.set_ylim([0, 150])
axis.set_xlim([0.2, 20])
axis.yaxis.grid(color='gray')
axis.set_title("Espectro Rádio" + str(horario_pico)[10:19])
plt.legend()

nome_do_arquivo = "espectro"
if platform.system() == "Linux":
    caminho = CAMINHO_ABSOLUTO + diretorio + "/" + nome_do_arquivo
else:
    caminho = CAMINHO_ABSOLUTO + diretorio + "\\" + nome_do_arquivo

fig_espectro.savefig(caminho)

## GOES

In [ ]:
comeco_flare = str(df.index[tempo1_flare])
fim_flare = str(df.index[tempo2_flare])

comeco_flare = comeco_flare[0:19].replace('-', '/')
fim_flare = fim_flare[0:19].replace('-', '/')

tr = TimeRange(comeco_flare, fim_flare)

results = Fido.search(attrs.Time(tr), attrs.Instrument("XRS"))
files = Fido.fetch(results)
goes = TimeSeries(files)
goes = goes.to_dataframe()
comeco, fim = get_correct_goes_index(goes.index, comeco_flare, fim_flare)
goes = goes[comeco:fim]

In [ ]:
fig, axis = plt.subplots()

# Usa o mesmo index do dataframe para o eixo x.
axis.semilogy(goes.index, goes["xrsa"], label=r"0.5--4.0 $\AA$", color="b", linewidth=2)
axis.semilogy(goes.index, goes["xrsb"], label=r"1.0--8.0 $\AA$", color="r", linewidth=2)
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
axis.set_title("GOES Xray Flux")
axis.set_axisbelow(True)
axis.yaxis.grid(color='gray')
axis.set_ylabel("Watts $m^{-2}$")
axis.set_xlabel(ano + "-" + mes + "-" + dia)
axis.set_ylim([1e-9, 1e-2])

plt.legend()

ax2 = axis.twinx()
ax2.set_yscale("log")
ax2.set_ylim(1e-9, 1e-2)
ax2.set_yticks((1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2))
ax2.set_yticklabels((' ', 'A', 'B', 'C', 'M', 'X', ' '))

fig_goes = ax2.get_figure()

nome_do_arquivo = "GOES"
if platform.system() == "Linux":
    fig_goes.savefig(CAMINHO_ABSOLUTO + diretorio + "/" + nome_do_arquivo)
else:
    fig_goes.savefig(CAMINHO_ABSOLUTO + diretorio + "\\" + nome_do_arquivo)

## Região Ativa

In [ ]:
comeco_flare = comeco_flare.replace('/', '-')
fim_flare = fim_flare.replace('/', '-')

caminho_eventos = "Eventos/" + ano + "_events/"
report = NoaaReport(ano, mes, dia, caminho_eventos)
report.get_dataframe()

In [ ]:
ar = "1035"

## Grau de polarização

In [ ]:
fig_7ghz, axis = plt.subplots()

# Usa o mesmo index do dataframe para o eixo x.
axis.plot(LM_normalizado, label="L")
axis.plot(RM_normalizado, label="R")
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

axis.legend()
axis.set_ylabel("Flux (SFU)")
axis.set_xlabel("Horário")
plt.title("Evento 7GHz")

fig_7ghz.canvas.mpl_connect('button_press_event', onclick)

filename = "grafico_normalizado"
if platform.system() == "Linux":
    fig_7ghz.savefig(CAMINHO_ABSOLUTO + diretorio + "/" + filename)
else:
    fig_7ghz.savefig(CAMINHO_ABSOLUTO + diretorio + "\\" + filename)

In [ ]:
posicao_inicial= num2date(posicao[-1][0])
indice_de_y1 = calculo_de_indice(df, posicao_inicial)

pico_R = max(df["R_normalizado"][tempo1_flare:tempo2_flare])
pico_L = max(df["L_normalizado"][tempo1_flare:tempo2_flare])

ponto_do_r_inicial = indice_de_y1[0][2]
ponto_do_r_inicial = float("{:.3f}".format(ponto_do_r_inicial))
print('Ponto do R inicial:', ponto_do_r_inicial)

ponto_do_r_no_pico = float(str(pico_R)[0:7])
ponto_do_r_no_pico = float("{:.3f}".format(ponto_do_r_no_pico))
print('Ponto do R no pico:', ponto_do_r_no_pico)

ponto_do_l_inicial = indice_de_y1[0][3]
ponto_do_l_inicial = float("{:.3f}".format(ponto_do_l_inicial))
print('Ponto do L incial:', ponto_do_l_inicial)

ponto_do_l_no_pico = float(str(pico_L)[0:7])
ponto_do_l_no_pico = float("{:.3f}".format(ponto_do_l_no_pico))
print('Ponto do L no pico:', ponto_do_l_no_pico)

# Procura o horario do pico.
for data in enumerate(df["R_normalizado"]):
    # Se encontrar o valor do pico, cria a variavel horario_do_pico.
    if data[1] == pico_R:
        horario_do_pico = df["R_normalizado"].index[data[0]]

tempo = str(horario_do_pico)[0:26]
print('Tempo:', tempo)

In [ ]:
I = ((ponto_do_r_no_pico - ponto_do_r_inicial) + (ponto_do_l_no_pico - ponto_do_l_inicial))
I = float("{:.3f}".format(I))
print("Cálculo do I:", I)

V = ((ponto_do_r_no_pico - ponto_do_r_inicial) - (ponto_do_l_no_pico - ponto_do_l_inicial))
V = float("{:.3f}".format(V))
print("Cálculo do V:", V)

In [ ]:
# Grau de polarização.
GP = (V / I) * 100
GP = float("{:.3f}".format(GP))
print('Grau de Polarização:', GP)

In [ ]:
# Geração de páginas

import matplotlib.gridspec as gridspec

ut = "Universal Time"

plt.rcParams['figure.figsize'] = 10,7

fig, axes = plt.subplots(3, 2)

if len(dia) == 1:
    dia = "0" + dia
if len(mes) == 1:
    mes = "0" + mes

gs = gridspec.GridSpec(6, 6)
gs.update(wspace=1.4, hspace=2)
ax1 = plt.subplot(gs[0:2, :3])
ax2 = plt.subplot(gs[2:4, :3])
ax3 = plt.subplot(gs[4:, :3])
ax4 = plt.subplot(gs[0:2, 3:])
ax5 = plt.subplot(gs[2::, 3:])

ax1.plot(LM_normalizado, label="L")
ax1.plot(RM_normalizado, label="R")
ax1.set_title("Event - 7 GHz - " + mes + "/" + dia + "/" + ano[2:])
ax1.set_ylabel("Flux (SFU)")
ax1.set_xlabel(ut)
ax1.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax1.yaxis.grid(color='gray')
ax1.legend()

# R+L
ax2.plot(RL_I)
ax2.set_ylabel("I [R+L] (SFU)")
ax2.set_xlabel(ut)
ax2.yaxis.grid(color='gray')
ax2.xaxis.set_major_formatter(DateFormatter('%H:%M'))

# R-L
ax3.plot(RL_V)
ax3.set_ylabel("V [R-L] (SFU)")
ax3.set_xlabel(ut)
ax3.yaxis.grid(color='gray')
ax3.xaxis.set_major_formatter(DateFormatter('%H:%M'))

# GOES
ax4.semilogy(goes.index, goes["xrsa"], label=r"0.5--4.0 $\AA$", color="b", linewidth=2)
ax4.semilogy(goes.index, goes["xrsb"], label=r"1.0--8.0 $\AA$", color="r", linewidth=2)
ax4.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax4.set_title("GOES Xray Flux")
ax4.set_axisbelow(True)
ax4.yaxis.grid(color='gray')
ax4.set_ylabel("Watts $m^{-2}$")
ax4.set_xlabel(ut)
ax4.set_ylim([10**-9, 10**-2])
ax4.legend()

ax4_2 = ax4.twinx()
ax4_2.set_yscale("log")
ax4_2.set_ylim(1e-9, 1e-2)
ax4_2.set_yticks((1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2))
ax4_2.set_yticklabels((' ', 'A', 'B', 'C', 'M', 'X', ' '))

# Espectro rádio
for column in picos_ordenados:
    ax5.plot(column, picos[str(column)], '-o', label=column)

ax5.set_ylabel("SFU")
ax5.set_xlabel("Frequency (GHz)")
ax5.set_ylim([-10, 120])
ax5.set_xlim([0.2, 20])
ax5.yaxis.grid(color='gray')
ax5.set_title("Espectro Rádio -" + str(horario_pico)[10:19])
ax5.legend()

filename = "pagina"
fig.savefig(CAMINHO_ABSOLUTO + diretorio + "\\" + filename)

In [ ]:
resultados_finais = np.transpose([[tempo], [ponto_do_r_inicial], [ponto_do_l_inicial],
                                  [ponto_do_r_no_pico], [ponto_do_l_no_pico], [GP],
                                  [I], [V], [str(ar)]])
a = pd.DataFrame(resultados_finais, columns = ['Time','PR','PL','PRP','PLP','GP','I','V', 'AR'])

if platform.system() == "Linux":
    a.to_csv(CAMINHO_ABSOLUTO + "/dados_finais/dados.csv", header=None, sep=' ', mode='a', index=False)
else:
    a.to_csv(CAMINHO_ABSOLUTO + "\\dados_finais\\dados.csv", header=None, sep=' ', mode='a', index=False)